# Data Cleaning
***
In this notebook the data was cleaned and prepared for EDA and modeling. Initially data types and statistical attributes were explored. Null values and duplicate rows were appropriately addressed. Then the text data was cleaned in the following order:
 - Transformed into lower case
 - Stripped of digits
 - Contractions expanded
 - Emojis transformed into words
 - Stripped of punctuation
 - Stripped of white space
 - Filtered from stop words
 - Lemmatized
 
Using the lang_detect library each review was tagged with an abbreviation of the language it was written in. Density features such as word/sentence count and different punctuation counts in a review were also created. Finally the textstat library is used to create even more features.
***

In [1]:
# Import libraries for data cleaning and text-preprocessing
import pandas as pd
import numpy as np

In [2]:
# Load data
gameReviews = pd.read_csv('MetacriticGameReviews.csv', index_col=0)
gameReviews.head()

,title,platform,metascore,metasentiment,average_userscore,average_usersentiment,developer,genre,number_of_players,esrb_rating,release_date,username,userscore,usersentiment,review,review_date
0,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,Up to 32,M,"Oct 26, 2018",gnadenlos,7,mixed,"The main problem is, that it's not a real open...","Nov 1, 2018"
1,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,Up to 32,M,"Oct 26, 2018",Feriatus,7,mixed,It's not a bad game but the gameplay is an out...,"Oct 29, 2018"
2,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,Up to 32,M,"Oct 26, 2018",ponux,7,mixed,"Visually superb (except cutscenes), good (not ...","Nov 5, 2018"
3,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,Up to 32,M,"Oct 26, 2018",Picklock,5,mixed,"Great looking game backed up by clumsy, overly...","Nov 4, 2018"
4,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,Up to 32,M,"Oct 26, 2018",Saints,6,mixed,Red Dead Redemption 2 is an amazing game that ...,"Oct 30, 2018"


In [3]:
#gameReviews.review[0]

### Initial data exploration
***

In [4]:
# Save shape of dataframe to compare after data is cleaned
init_shape = gameReviews.shape

# Check dtypes of each feature
gameReviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21075 entries, 0 to 21074
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   title                  21075 non-null  object 
 1   platform               21075 non-null  object 
 2   metascore              21045 non-null  float64
 3   metasentiment          21045 non-null  object 
 4   average_userscore      21045 non-null  float64
 5   average_usersentiment  21045 non-null  object 
 6   developer              21045 non-null  object 
 7   genre                  21060 non-null  object 
 8   number_of_players      17595 non-null  object 
 9   esrb_rating            20565 non-null  object 
 10  release_date           21060 non-null  object 
 11  username               21075 non-null  object 
 12  userscore              21075 non-null  int64  
 13  usersentiment          21075 non-null  object 
 14  review                 21073 non-null  object 
 15  re

***
>Many of the features are of the object type including the release and review date which may be of better us as time series type. There appears to be a significant amount of null values for the "number_of_players" column and a small amount of nulls for a few others.
***

In [5]:
# Checkout basic statistical qualities for numerical features
gameReviews.describe().T

,count,mean,std,min,25%,50%,75%,max
metascore,21045.0,75.761226,11.006157,17.0,70.0,78.0,83.0,97.0
average_userscore,21045.0,6.763792,1.618301,0.2,6.1,7.2,7.9,9.6
userscore,21075.0,6.769300,3.425507,0.0,4.0,8.0,10.0,10.0


***
>The average critic score tends to be around 76 which appear to be on a 10:1 scale compared to user scores. The average user scores and individual user scores both match around a score of 6.8.
***

### Dealing with nulls and duplicate rows
***

In [6]:
# There appears to be some missing values, check for percentage of missing values in each column
def missingData(df): # Function retrieved from github.com/ithisted/PetAdoptionPrediction/blob/master/PetAdoptionPrediction.ipynb
    missing = False
    
    for item in (df.isnull().sum()).iteritems():
        if item[1] > 0:
            print('Missing Data percentage for '+item[0]+' is {:2.2%}'.format((item[1]/df.shape[0])) )
            missing = True
    if not missing:
        print('Found no missing values.')

# Check which columns have nulls and what percent of nulls are present
missingData(gameReviews)

Missing Data percentage for metascore is 0.14%
Missing Data percentage for metasentiment is 0.14%
Missing Data percentage for average_userscore is 0.14%
Missing Data percentage for average_usersentiment is 0.14%
Missing Data percentage for developer is 0.14%
Missing Data percentage for genre is 0.07%
Missing Data percentage for number_of_players is 16.51%
Missing Data percentage for esrb_rating is 2.42%
Missing Data percentage for release_date is 0.07%
Missing Data percentage for review is 0.01%


In [7]:
# Drop rows of missing values of columns with 2% or less of nulls
def dropMissing(df, drop_list):
    df = df.dropna(axis=0, subset=drop_list)
    return df

# Initialize a list of column names to drop the nulls from
toDrop = ['metascore', 'metasentiment', 'average_userscore', 'average_usersentiment', 'developer', 'esrb_rating','review']

# Call the function and check again for missing values, should only be 'number_of_players'
gameReviews = dropMissing(gameReviews, toDrop)
missingData(gameReviews)

Missing Data percentage for number_of_players is 15.93%


In [8]:
# Explore different values of 'number_of_players column'
gameReviews.number_of_players.value_counts(dropna=False)

No Online Multiplayer    8025
NaN                      3270
Up to 4                  1860
2                        1559
Up to 8                  1140
Up to 10                  705
Up to 6                   645
Up to 12                  570
Up to 16                  434
Up to 22                  390
Online Multiplayer        330
Up to 5                   225
Up to 18                  195
Up to 24                  180
Up to 64                  165
Up to more than 64        165
Up to 3                   150
Massively Multiplayer     150
Up to 20                  105
Up to 32                   75
Up to 40                   75
Up to 30                   45
Up to 60                   45
1 Player                   30
Name: number_of_players, dtype: int64

***
>To address the null values in the number_of_players column, the titles for the games having a null value were explored. There were a mixture of singleplayer and multiplayer games missing values for the number of players. Each title was googled to see if the game was a single or multiplayer game. 
<br><br>
Then, to reduce the amount of different, uneccessary amount of values for the 'number_of_players' columns of the dataframe, the game titles for 'No Online Multiplayer' games were also explored. Mostly all, if not all were single player games. For simplicity, the 'number_of_players' columns is then converted to a binary columnn where a game is either a single player game (values of 'No Online Multiplayer' and '1 Player) or multiplayer game (all other values).
***

In [9]:
# Explore titles of null 'number_of_players' values
titles_of_null_nop = pd.unique(gameReviews[gameReviews.number_of_players.isnull()]['title'])
#titles_of_null_nop

In [10]:
# Initialize a list with all multiplayer games
multiplayer = ['NBA 2K17',
              'Call of Duty: Modern Warfare Remastered',
               'Destiny: The Taken King',
               'DiRT Rally',
               'Killer Instinct: Definitive Edition',
               'Destiny 2: Forsaken',
               'Project CARS 2',
               'FIFA 18',
               'Sonic Mania',
               'Child of Light',
               'Brothers: A Tale of Two Sons',
               'Guitar Hero Live',
               'NHL 18',
               'Peggle 2',
               'World of Tanks',
               'Need for Speed: Rivals',
               'Resident Evil: Revelations 2',
               'Project Spark',
               'Titanfall: Expedition',
               'Phantom Dust',
               'Madden NFL 25',
               "Mirror's Edge Catalyst",
               'Destiny: House of Wolves',
               'Never Alone',
               'Powerstar Golf',
               'The Golf Club',
               'The Crew',
               'Resident Evil 6',
               'Crimson Dragon',
               'Goat Simulator',
               'Halo: Spartan Assault',
               'Ghostbusters',
               'XCOM 2: War of the Chosen',
               'Shovel Knight',
               'Monster Hunter: World - Iceborne',
               'Nex Machina: Death Machine',
               'Guacamelee! Super Turbo Championship Edition',
               'Bloodborne: The Old Hunters',
               'TowerFall Ascension',
               'OlliOlli2: Welcome to Olliwood',
               'Super Mega Baseball',
               'MLB The Show 16',
               'Pyre',
               'Resogun',
               'N++',
               'Dark Souls III: The Ringed City',
               'Enter the Gungeon',
               'PixelJunk Shooter Ultimate',
               'ZEN Pinball 2',
               'Pix the Cat',
               'Injustice: Gods Among Us - Ultimate Edition',
               'MLB 15: The Show',
               'The Banner Saga',
               'Overcooked!',
               "Don't Starve: Console Edition",
               'PlanetSide 2',
               'Dying Light: The Following',
               'Apotheon',
               'Mortal Kombat 11: Aftermath',
               'Dead Nation: Apocalypse Edition',
               "Assassin's Creed IV: Black Flag - Freedom Cry",
               'Dark Souls III: Ashes of Ariandel',
               'Zombi',
               'Dead Star',
               'Knack 2',
               'Trove',
               'Destiny: The Dark Below',
               'Loadout',
               'Sniper: Ghost Warrior 3',
               'Knack',
               'Rayman Legends: Definitive Edition',
               'Fast RMX',
               'Blaster Master Zero',
               'Overcooked!: Special Edition',
               'NBA Playgrounds',
               'Astro Bears Party'
              ]

In [11]:
# Replace all null values with either multiplayer or singleplayer
for t in titles_of_null_nop:
    if t in multiplayer:
        gameReviews.loc[gameReviews.title == t, 'number_of_players'] = 'multiplayer'
    else:
        gameReviews.loc[gameReviews.title == t, 'number_of_players'] = 'singleplayer'

In [12]:
# Explore titles of 'No Online Multiplayer' games
#pd.unique(gameReviews[gameReviews.number_of_players == 'No Online Multiplayer']['title'])

In [13]:
# Retrieve all unique values in 'number_of_players' column and seperate which will be considered as single and multi player
num_players_values = pd.unique(gameReviews.number_of_players)
single = ['No Online Multiplayer', '1 Player', 'singleplayer']
multi = [val for val in num_players_values if val not in single]

# Replace corresponding values to get a binary column
gameReviews['number_of_players'] = gameReviews.number_of_players.replace(single, 'singleplayer')
gameReviews['number_of_players'] = gameReviews.number_of_players.replace(multi, 'multiplayer')

# Double check only two unique values exists; singleplayer and multiplayer
gameReviews.number_of_players.value_counts()

multiplayer     10543
singleplayer     9990
Name: number_of_players, dtype: int64

***
>After addressing all missing values, duplicate rows, if any, were eliminated from the data.
***

In [14]:
# Check for duplicate rows and drop if any
def dropDuplicates(df):
    num_dups = len(df) - len(df.drop_duplicates())
    print(num_dups, "duplicate rows were dropped.")
    return df.drop_duplicates()

# Call the function to drop duplicates
gameReviews = dropDuplicates(gameReviews)

# Reset index as rows of missing and duplicate values have been dropped
gameReviews.reset_index(drop=True, inplace=True)

0 duplicate rows were dropped.


In [15]:
# Compare shape of clean dataframe to the initiail shape
clean_shape = gameReviews.shape
print('Initial shape:', init_shape)
print('Current shape:', clean_shape)

Initial shape: (21075, 16)
Current shape: (20533, 16)


### Text cleaning
***

In [16]:
# Import libraries for text-processing
from contractions import CONTRACTION_MAP
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import emoji
import string
import re

#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('stopwords')

In [17]:
def StopWords(negation=True):
    '''Initializes stop words from nltk package with the option to remove negation words from stop word list'''
    # Initialize stopwords
    stop_words = stopwords.words('english')
    # Add domain specific words to the list of stop words
    stop_words.append('video')
    stop_words.append('game')
    
    if negation:
        # Remove negation words to extract correct sentiment
        stop_words.remove('no')
        stop_words.remove('not')
        
    return stop_words

In [18]:
def make_lower(docs):
    '''Transforms text into lower case'''
    lowered = [doc.lower() for doc in docs]
    return lowered

In [19]:
def remove_digits(docs):
    '''Removes digits from text'''
    pattern = r'\d*'
    digitless = [re.sub(pattern, '', doc) for doc in docs]
    return digitless

In [20]:
# Function retrieved from https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72
def expand_contractions(docs, contraction_mapping=CONTRACTION_MAP):
    '''Expands contractions from a predefined list of common contractions to seperate negation words
    from stop words'''
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    
    expanded_texts = []
    for doc in docs:
        expanded_text = contractions_pattern.sub(expand_match, doc)
        expanded_text = re.sub("'", "", expanded_text)
        expanded_texts.append(expanded_text)
        
    return expanded_texts

In [21]:
def emoji_to_words(docs):
    '''Transforms emojis to text using emoji library'''
    demojized = [emoji.demojize(doc) for doc in docs]
    return demojized

In [22]:
def remove_punctuation(docs):
    '''Removes punctuation from text, for hyphens and '/' marks words are seperated then the marks are removed'''
    punc_filtered = []
    
    # Seperate punctuation marks that are in between words from those that occur at the end of words
    punctuation = ''.join([p for p in string.punctuation if p not in ['/','-','_']])
    punctuation = punctuation + '’'
    between_words = r'[/\-\_]'
    
    # Translate punctuation marks end of word punctuation marks with ''
    table = str.maketrans("","",punctuation)
    for doc in docs:
        no_puncs = re.sub(between_words, ' ', doc)
        no_puncs = no_puncs.translate(table)
        punc_filtered.append(no_puncs)
    
    #no_puncs = [doc.translate(table) for doc in docs]
    return punc_filtered

In [23]:
def remove_whitespace(docs):
    '''Removes extra white space from text'''
    no_ws = [' '.join(doc.split()) for doc in docs]
    return no_ws

In [24]:
def remove_stopwords(docs):
    ''' Removes english stop words with the addition of 'game' and 'video', considered as domain specific stop words'''
    stopword_filtered = []

    # Initialize stopwords
    stop_words = StopWords()

    for doc in docs:
        tokens = word_tokenize(doc)
        output = [t for t in tokens if t not in stop_words]
        stopword_filtered.append(' '.join(output))
  
    return stopword_filtered

In [25]:
def lemmatize(docs):
    '''Converts words to their lemma form'''
    # Initialize empty string to store lemmatized text
    lemmatized = []
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # Loop over all text files, tokenize each words, get its lemma and join lemmas to create lemmatized strings
    for doc in docs:
        tokens = word_tokenize(doc)
        output = [lemmatizer.lemmatize(t) for t in tokens]
        lemmatized.append(' '.join(output))
    
    return lemmatized

In [26]:
def text_process(df, col, lower=True, contraction=True, digit=True, emoji=True, punctuation=True, whitespace=True, stopwords=True, lemma=True):
    '''Processes a corpus of documents, allowing the user to specify how to clean text data'''
    clean_corpus = list(df[col])
    
    if lower:
        clean_corpus = make_lower(clean_corpus)
        
    if contraction:
        clean_corpus = expand_contractions(clean_corpus)
        
    if digit:
        clean_corpus = remove_digits(clean_corpus)
        
    if emoji:
        clean_corpus = emoji_to_words(clean_corpus)
        
    if punctuation:
        clean_corpus = remove_punctuation(clean_corpus)
        
    if whitespace:
        clean_corpus = remove_whitespace(clean_corpus)

    if stopwords:
        clean_corpus = remove_stopwords(clean_corpus)
        
    if lemma:
        clean_corpus = lemmatize(clean_corpus)
        
    return clean_corpus

In [27]:
# Call function to clean the reviews to prepare them for a word vectorizer
gameReviews['clean_text'] = text_process(gameReviews, 'review')

In [28]:
# Compare a raw review to its clean counterpart
print(gameReviews.review[0], '\n')
print(gameReviews.clean_text[0])

The main problem is, that it's not a real open world game. If you focus on the main story, like many reviewers and some users do, you will experience a linear and scripted game with almost no freedom. Every time you try something different the missions will fail. Controls aren't very good, so it's also hard to recommend the game for that linear story experience. The separate and real open world part is done quite well, but it doesn't have enough interesting and coherent/interlocking elements to keep you motivated very long. If you remove the linear story and high production value, it's the usual open world stuff, we've known for years, mixed with light survival elements. So what you will get is an average to good open world sandbox for maybe 15-20 hours of random fun and a separate, very scripted and cinematic game with high production value. Both parts have average gameplay and problematic controls. The only highlights and reasons to play the game are its great graphics, atmosphere an

***
>A raw version of the first game review for the game Red Dead Redemption 2 was compared to one that had been cleaned. The reviewer enjoys the atmosphere and enviroment of the world in Red Dead Redemption. However, they felt that the game's primary plot was too linear and not flexible in the options it provided the player to overcome major obstacles. When reading the clean version of the review, one can still extract the reviewers sentiment from the original review.
***

### Feature engineering
***

#### - Label reviews by language

In [29]:
# Import function to determine the language of the reviews
from langdetect import detect

# Loop over all reviews and create a new feature 'language'
language = []
for i in range(len(gameReviews.clean_text)):
    try:
        lang = detect(gameReviews.clean_text[i])
    except:
        lang = 'Error'
    
    language.append(lang)
        
gameReviews['language'] = language

***
>Some of the languages for the reviews could not be determined. With the 'try', 'except' functions those reviews were labeled as 'Error' making them easy to find and check why the detect function failed to assign a language code to them.
The problematic reviews were mostly composed of characters and digits. These reviews were all in english or characters, as such they were labeled as english.
***

In [30]:
# Filter out the errors where the language could not be determined
language_error = gameReviews[gameReviews['language'] == 'Error']
print(len(language_error))

5


In [31]:
# Print out all the problematic reviews as there were only 5
for r in language_error.review:
    print(r)

+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
the same game 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,29,30,31,32,33,34,35,36
A+    +
10/10 -------------------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------


In [32]:
# Remove rows where reviews were unusable
gameReviews = gameReviews[gameReviews.language != 'Error']

# Explore all languages present in the set of reviews
# Language code abbreviations: https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes
#gameReviews.language.value_counts(dropna=False)

#### - Density features

In [33]:
# Create a number of words and sentences feature
clean_words = text_process(gameReviews, 'review', stopwords=False, lemma=False)
gameReviews['word_count'] = [len(word_tokenize(text)) for text in clean_words]
gameReviews['sentence_count'] = [len(sent_tokenize(text)) for text in gameReviews.review]
gameReviews['exclamation_count'] = [text.count('!') for text in gameReviews.review]
gameReviews['question_count'] = [text.count('?') for text in gameReviews.review]
gameReviews['period_count'] = [text.count('.') for text in gameReviews.review]

#### - Textstat features

In [34]:
import textstat

# Create a feature for reading level of reviews and syllable count
gameReviews['reading_level'] = gameReviews.review.apply(textstat.flesch_reading_ease)
gameReviews['syllable_count'] = gameReviews.review.apply(textstat.syllable_count)

#### - POS features

In [35]:
# Create a column of clean words tagged with their POS
gameReviews['POS_tagged'] = [pos_tag(word_tokenize(text)) for text in clean_words]

# Initialize different POS variables to count as features
conjunction = 'CC'
adjectives = ['JJ', 'JJR', 'JJS']
nouns = ['NN', 'NNS', 'NNP', 'NNPS']
pronouns = ['PRP', 'PRP$', 'WP', 'WP$']
adverbs = ['RB', 'RBR', 'RBS', 'WRB']
verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

# Initialize empty lists to add as features to df
conj = []
adj = []
nn = []
pnn = []
adv = []
vrb = []
other_pos = []

# Loop over POS tagged words to retrieve counts of POS of interest
for text in gameReviews.POS_tagged:
    # Set count variables to track POS in the reviews
    conj_cnt = 0
    adj_cnt = 0
    nn_cnt = 0
    pnn_cnt = 0
    adv_cnt = 0
    vrb_cnt = 0
    others_cnt = 0
    
    # Count different POS for extra features
    for tpl in text:
        pos = tpl[1]
        if pos == conjunction:
            conj_cnt += 1
        elif pos in adjectives:
            adj_cnt += 1
        elif pos in nouns:
            nn_cnt += 1
        elif pos in pronouns:
            pnn_cnt += 1
        elif pos in adverbs:
            adv_cnt += 1
        elif pos in verbs:
            vrb_cnt += 1
        else:
            others_cnt += 1
    
    # Append POS counts to corresponding lists
    conj.append(conj_cnt)
    adj.append(adj_cnt)
    nn.append(nn_cnt)
    pnn.append(pnn_cnt)
    adv.append(adv_cnt)
    vrb.append(vrb_cnt)
    other_pos.append(others_cnt)

In [36]:
# Add POS count features to df
gameReviews['conjunction_count'] = conj
gameReviews['adjective_count'] = adj
gameReviews['noun_count'] = nn
gameReviews['pronoun_count'] = pnn
gameReviews['adverb_count'] = adv
gameReviews['verb_count'] = vrb
gameReviews['other_POS_count'] = other_pos

In [37]:
gameReviews.head()

,title,platform,metascore,metasentiment,average_userscore,average_usersentiment,developer,genre,number_of_players,esrb_rating,...,reading_level,syllable_count,POS_tagged,conjunction_count,adjective_count,noun_count,pronoun_count,adverb_count,verb_count,other_POS_count
0,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,multiplayer,M,...,60.55,254,"[(the, DT), (main, JJ), (problem, NN), (is, VB...",11,35,43,13,13,25,38
1,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,multiplayer,M,...,60.69,118,"[(it, PRP), (is, VBZ), (not, RB), (a, DT), (ba...",4,6,22,4,2,18,23
2,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,multiplayer,M,...,60.89,495,"[(visually, RB), (superb, JJ), (except, IN), (...",19,41,83,28,35,65,80
3,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,multiplayer,M,...,76.15,459,"[(great, JJ), (looking, VBG), (game, NN), (bac...",9,28,73,25,37,81,99
4,Red Dead Redemption 2,Xbox One,97.0,positive,7.8,positive,Rockstar Games,Action adventure,multiplayer,M,...,67.49,820,"[(red, JJ), (dead, JJ), (redemption, NN), (is,...",26,44,128,35,72,132,177


In [38]:
# Export data for EDA and modeling
gameReviews.to_csv('MetacriticGameReviewsClean.csv')